In [1]:
import pandas as pd
import numpy_financial as npf
import numpy as np
import datetime
from re import sub
from decimal import Decimal

from config import getConfig
from calcIRR import calcIRR

In [2]:
def formatValue(x):
    return Decimal(sub(r'[^\d.]', '', x))

In [3]:
def getDays(df):
    startDate = acctValues.iloc[-1]['date']
    startDate = pd.to_datetime(startDate).date()
    endDate = acctValues.iloc[0]['date']
    endDate = pd.to_datetime(endDate).date()

    diff = datetime.date.today()-startDate
    print("{:,} days between {} and {}".format(diff.days, startDate, endDate))
    return diff.days

In [4]:
cfg = getConfig()
loc = cfg['ETrade']['dataLoc']

In [5]:
cfg

{'ETrade': {'dataLoc': '~/data/investing/ETrade/',
  'fileName': 'accountValues.csv'},
 'Fisher': {'dataLoc': '~/data/investing/Fisher/'},
 'SPX': {'dataLoc': '~/data/investing/benchmark/', 'fileName': 'spx.csv'},
 'MSCI': {'dataLoc': '~/data/investing/benchmark/', 'fileName': 'msci.csv'},
 'marketAPI': {'URL': 'http://api.marketstack.com/v1/',
  'APIKey': '?access_key=ffa68edbfb08de7589a43751795e0e07'}}

In [5]:
deposits = pd.read_csv(loc+'deposits.csv')
acctValues = pd.read_csv(loc+'accountValues.csv',
                         usecols=['date', 'value'])

In [8]:
# Remove $ sign from account values
acctValues['value'] = acctValues['value'].apply(lambda x: formatValue(x))

In [10]:
# Make sure the starting dates are matched between account values and deposits
depStartDt = deposits['date'].min()
acctStartDt = acctValues['date'].min()

startDt = depStartDt if depStartDt > acctStartDt else acctStartDt

deposits = deposits[deposits['date'] >= startDt]
acctValues = acctValues[acctValues['date'] >= startDt]

#deposits['date'].min(), acctValues['date'].min()

In [14]:
deposits['amount'].sum()

50000

In [13]:
# There's 1 deposit more recent than last account value so remove it
deposits = deposits[1:]

In [13]:
# The 'initial account value' is basically the first deposit so add a row for it
d = {}
acctValues.sort_values('date', ignore_index=True, inplace=True)
d['amount'] = acctValues.iloc[0]['value']
d['date'] = acctValues.iloc[0]['date']

deposits.loc[len(deposits)] = d
deposits.sort_values('date', ignore_index=True, inplace=True)

In [14]:
# Ending value and date
endValue = acctValues.iloc[-1]['value']
endDate = acctValues.iloc[-1]['date']

endDate = pd.to_datetime(endDate).date()

In [15]:
irr = calcIRR(deposits, endDate, endValue, 100)

Total: 242,821



In [16]:
# Growth of 10k over a year at the computed rate
final = npf.fv(irr, nper=365, pmt=0, pv=-1e4)
print("Annualized rate: {:.2%}".format(final/1e4 -1))

Annualized rate: 9.76%
